In [1]:
!pip install hyperas

In [2]:
import h5py
from os.path import join,exists
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.optimizers import RMSprop
from hyperas.distributions import choice
from hyperas import optim
from keras.callbacks import ModelCheckpoint
from keras.constraints import maxnorm
from random import randint
from keras import backend as K
import os
import numpy as np
from sklearn.utils import shuffle
from hyperopt import Trials, STATUS_OK, tpe
import tensorflow as tf
K.set_image_data_format('channels_first')

In [3]:
os.chdir('/content/drive/My Drive/Colab Notebooks/synopsys-2022/models/regression')

In [4]:
def create_model(X_train, Y_train, X_test, Y_test):
    W_maxnorm = 3
    DROPOUT = {{choice([0.3,0.5,0.7])}}
    # DROPOUT = 0.3

    model = Sequential()
    model.add(Conv2D(64, (1, 5), padding='same', input_shape=(20, 1, 20),activation='relu',kernel_constraint=maxnorm(W_maxnorm)))
    model.add(MaxPool2D(pool_size=(1, 2),strides=(1,2)))
    model.add(Conv2D(128, (1, 5), padding='same', activation='relu',kernel_constraint=maxnorm(W_maxnorm)))
    model.add(MaxPool2D(pool_size=(1, 2),strides=(1,2)))
    model.add(Flatten())

    model.add(Dense(16,activation='relu',kernel_constraint=maxnorm(W_maxnorm)))
    model.add(Dropout(DROPOUT))
    #model.add(Dense(2,W_constraint=maxnorm(W_maxnorm)))
    #model.add(Activation('softmax'))
    model.add(Dense(1))

    myoptimizer = RMSprop(lr={{choice([0.01,0.001,0.0001,1e-5,1e-1])}}, rho=0.9, epsilon=1e-06)
    # myoptimizer = RMSprop(lr=0.01, rho=0.9, epsilon=1e-06)
    #mylossfunc = 'binary_crossentropy'
    mylossfunc='mean_squared_error'
    model.compile(loss=mylossfunc, optimizer=myoptimizer)
    # print(model.summary())

    result = model.fit(X_train, Y_train, batch_size=100, epochs=5,validation_split=0.1,verbose=False)

    val_loss = np.amax(result.history['val_loss'])
    # model_arch = 'MODEL_ARCH'
    # bestaccfile = join('TOPDIR',model_arch,model_arch+'_hyperbestacc')
    # reportAcc(acc,score,bestaccfile)

    return {'loss': val_loss, 'status': STATUS_OK, 'model': model}

In [5]:
def train_data():
  data_train = h5py.File('./data/train.h5.batch1','r')
  X_train = np.array(data_train['data'])
  Y_train = np.array(data_train['label'])
  X_train_shuffled, Y_train_shuffled = shuffle(X_train, Y_train)

  data_test = h5py.File('./data/test_new.h5.batch1', 'r')
  X_test = np.array(data_test['data'])
  Y_test = np.array(data_test['label'])
  return X_train_shuffled, Y_train_shuffled, X_test, Y_test

In [6]:
best_run, best_model = optim.minimize(model=create_model,
                                          data=train_data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='seq_32x2_16',
                                          verbose=False)

  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)



 40%|████      | 4/10 [01:06<01:39, 16.52s/it, best loss: 0.30278778076171875]


KeyboardInterrupt: ignored

In [ ]:
X_train, Y_train, X_test, Y_test = train_data()
best_model.fit(X_train, Y_train, batch_size=100, epochs=20,validation_split=0.1,verbose=True)

In [ ]:
best_model.evaluate(X_test, Y_test)

In [ ]:
best_model.save('./seq_32x2_16')